In [1]:
import os
import datetime
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import QuantLib as ql


In [2]:
from quant_lib.swap_curve import get_quote, swap_curve

In [24]:
class FRA():
    def __init__(self, today, pricing_date, maturity, position, fra_rate, notional):
        
        # initial setup
        self.date = today
        # 아마 아래는 today를 넣는게 맞을 것 같은데
        self.cuve = self.curve(self.date)

        self.pricing_date = ql.Date(pricing_date.day, pricing_date.month, pricing_date.year)
        self.maturity_date = ql.Date(maturity.day, maturity.month, maturity.year)

        self.fra_rate = fra_rate
        self.notional = notional

        if position == 'long' or position == 'Long':
            self.position = ql.Position.Long
        else:
            self.postion = ql.Position.Short

        # pricing result
        self.npv = self.pricing(self.curve)
        self.delta = self.DELTA()
        self.theta = self.THETA()
    
    def curve(self, date):
        return swap_curve(date, get_quote(date))
    

    def pricing(self, curve):
        # this method calculate FRA's theoritical price with FRA module in quantlib
        # before floating rate is determined, price would fluctuate.
        # so libor rate object is needed. In quantlib library, liborIndex class enables you to calculate underlying rate object

        #yield term structure
        curve_handle = ql.YieldTermStructureHandle(curve)

        # floating rate
        libor = ql.USDLibor(ql.Period(3, ql.Months), curve_handle)

        # pricing FRA
        fra = ql.ForwardRateAgreement(
                                    self.pricing_date,
                                    self.maturity_date,
                                    self.position,
                                    self.fra_rate,
                                    self.notional.
                                    libor,
                                    curve_handle
                                    )

        # fra price
        npv = fra.NPV()
        
        return npv

    
    def DELTA(self):
       # delta is change in values if 1bp of interest rate curve changes
        curve_handle = ql.YieldTermStructureHandle(self.curve)

        basis_point = 0.0001

        # fra price when 1bp up
        up_curve = ql.ZeroSpreadedTermStructure(
                                                curve_handle,
                                                ql.QuoteHandle(ql.SimpleQuote(basis_point))
                                                )
        up_fra = self.pricing(up_curve)

        down_curve = ql.ZeroSpreadedTermStructure(
                                                curve_handle,
                                                ql.QuoteHandle(ql.SimpleQuote(-basis_point))
                                                )
        down_fra = self.pricing(down_curve)

        #DV01 
        dv01 = (up_fra - down_fra)/2
        
        return dv01

    def THETA(self):
        # theta is change in value if one unit time passes.
        # since derivative product have time value, time to maturity is major variable in pricing derivatives
        price_t0 = self.pricing(self.curve(self.date))
        price_t1 = self.pricing(self.date + datetime.timedelta(days=1))

        return price_t1 - price_t0

In [25]:
todays_date = datetime.date(2021, 10, 9)
pricing_date = datetime.date(2022, 1, 9)
maturity_date = datetime.date(2022, 4, 9)

position = 'long'
fra_rate = 0.0022
notional = 1000000

fra = FRA(
    today=todays_date,
    pricing_date=pricing_date,
    maturity=maturity_date,
    position=position,
    fra_rate=fra_rate,
    notional=notional
)

TypeError: Wrong number or type of arguments for overloaded function 'new_YieldTermStructureHandle'.
  Possible C/C++ prototypes are:
    Handle< YieldTermStructure >::Handle(ext::shared_ptr< YieldTermStructure > const &)
    Handle< YieldTermStructure >::Handle()


In [10]:
print("price of FRA = {}".format(fra.npv))
print("delta of FRA = {}".format(fra.delta))
print("theta of FRA = {}".format(fra.theta))

price of FRA = <bound method FRA.pricing of <__main__.FRA object at 0x7f3d43de2f10>>
delta of FRA = <bound method FRA.delta of <__main__.FRA object at 0x7f3d43de2f10>>
theta of FRA = <bound method FRA.theta of <__main__.FRA object at 0x7f3d43de2f10>>
